# Automação Web e Busca de Informações com Python

#### Desafio: 

Trabalhamos em uma importadora e o preço dos nossos produtos é vinculado a cotação de:
- Dólar
- Euro
- Ouro

Precisamos pegar na internet, de forma automática, a cotação desses 3 itens e saber quanto devemos cobrar pelos nossos produtos, considerando uma margem de contribuição que temos na nossa base de dados.

Base de Dados: https://drive.google.com/drive/folders/1KmAdo593nD8J9QBaZxPOG1yxHZua4Rtv?usp=sharing

Para isso, vamos criar uma automação web:

- Usaremos o selenium
- Importante: baixar o webdriver

In [46]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys


navegador = webdriver.Chrome()
# Passo 2 - Cotação Dollar
navegador.get('https://www.google.com.br/')
# Pesquisar cotação Dollar
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Dólar')
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# Pegar o número que aparece no resultado da pesquisa.
cotacao_dolar = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_dolar)
navegador.get('https://www.google.com.br/')

# Passo 3 - Cotação Euro
navegador.get('https://www.google.com.br/')
# Pesquisar cotação Euro
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('Cotação Euro')
navegador.find_element_by_xpath(
    '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
# Pegar o número que aparece no resultado da pesquisa.
cotacao_euro = navegador.find_element_by_xpath(
    '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')
print(cotacao_euro)

#Cotação do Ouro
navegador.get('https://www.melhorcambio.com/ouro-hoje')
cotacao_ouro = navegador.find_element_by_xpath('//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = cotacao_ouro.replace(',', '.') # trocar vírgula pelo ponto.
print(cotacao_ouro)

navegador.quit() #fechar navegador



5.239
6.17913855
299.37


### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [47]:
#Passo 5
import pandas as pd

tabela = pd.read_excel('Produtos.xlsx')
display(tabela)



,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [48]:
#atualizar a cotação da moeda colocando a coluna 'Moeda' == Dólar/Eur/Ouro 

tabela.loc[tabela['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
tabela.loc[tabela['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
tabela.loc[tabela['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#atualizar o preço de compra >>> preço original x cotação
tabela['Preço Base Original'] = tabela['Preço Base Reais']* tabela['Cotação']

#preço de venda >>> preço de compra x margem
tabela['Preço Final'] = tabela['Preço Base Reais']*tabela['Margem']

#Arredondamento de valores para duas casas decimais.
tabela['Cotação'] = tabela['Cotação'].map('{:.2f}'.format)
tabela['Preço Base Original'] = tabela['Preço Base Original'].map('{:.2f}'.format)
tabela['Preço Final'] = tabela['Preço Final'].map('{:.2f}'.format)

display(tabela)

,Produtos,Preço Base Original,Moeda,Cotação,Preço Base Reais,Margem,Preço Final
0,Câmera Canon,26194.74,Dólar,5.24,4999.95,1.40,6999.93
1,Carro Renault,166836.74,Euro,6.18,27000.00,2.00,54000.00
2,Notebook Dell,23575.24,Dólar,5.24,4499.95,1.70,7649.91
3,IPhone,20929.81,Dólar,5.24,3995.00,1.70,6791.50
4,Carro Fiat,111224.49,Euro,6.18,18000.00,1.90,34200.00
5,Celular Xiaomi,12586.17,Dólar,5.24,2402.40,2.00,4804.80
6,Joia 20g,2095590.00,Ouro,299.37,7000.00,1.15,8050.00


### Agora vamos exportar a nova base de preços atualizada

In [49]:
#Passo 6 
#Passar com o mesmo vai substituir o arquivo, criar um novo nome é um novo arquivo.
tabela.to_excel('Produtos_Novo.xlsx', index=False)